**Title**: Download CVE data\
**Description**: Notebook to download all CVE data\
**Developer**: Teck Lim\
**Create date**: 03/28/2021

In [13]:
!pip install requests==2.25.1

Defaulting to user installation because normal site-packages is not writeable


In [11]:
import requests
import json
import time

In [9]:
file_path = '../../data/raw/cve.json'

In [3]:
base_url = 'https://services.nvd.nist.gov/rest/json/cves/1.0'
page_size = 1000
sleep_duration = 3

In [4]:
def scrap_cve(total_page=1000, start_page=0):
    cve_items = list()
    for page_no in range(start_page, total_page):
        for attempt in range(5):
            try:
                print('Retriving page: {}'.format(page_no+1))
                url = '{}?startIndex={}&resultsPerPage={}'.format(base_url, page_no*page_size, page_size)
                response = requests.get(url)
                response_json = response.json()
                break
            except:
                print('Something is wrong. Sleep for {} sec before retrying'.format(sleep_duration))
                time.sleep(sleep_duration)
        else:
            raise BaseException('Exhausted all attempts')

        cve_items += response_json['result']['CVE_Items']          
        print('Total scrapped: {}'.format(len(cve_items)))
        if len(cve_items) + (start_page * total_page) == response_json['totalResults']:
            print('Completed scrapping..sleep_duration')
            break
        time.sleep(sleep_duration)
    return cve_items

In [5]:
items = scrap_cve()
data = json.dumps(items)
with open(file_path, 'w') as fp:
    fp.write(data)

Retriving page: 1
Total scrapped: 1000
Retriving page: 2
Total scrapped: 2000
Retriving page: 3
Total scrapped: 3000
Retriving page: 4
Total scrapped: 4000
Retriving page: 5
Total scrapped: 5000
Retriving page: 6
Total scrapped: 6000
Retriving page: 7
Total scrapped: 7000
Retriving page: 8
Total scrapped: 8000
Retriving page: 9
Total scrapped: 9000
Retriving page: 10
Total scrapped: 10000
Retriving page: 11
Total scrapped: 11000
Retriving page: 12
Total scrapped: 12000
Retriving page: 13
Total scrapped: 13000
Retriving page: 14
Total scrapped: 14000
Retriving page: 15
Total scrapped: 15000
Retriving page: 16
Total scrapped: 16000
Retriving page: 17
Total scrapped: 17000
Retriving page: 18
Total scrapped: 18000
Retriving page: 19
Total scrapped: 19000
Retriving page: 20
Total scrapped: 20000
Retriving page: 21
Total scrapped: 21000
Retriving page: 22
Total scrapped: 22000
Retriving page: 23
Total scrapped: 23000
Retriving page: 24
Total scrapped: 24000
Retriving page: 25
Total scrapped:

In [6]:
len(items)

150600

In [12]:
with open(file_path, 'r') as fp:
    data = json.load(fp) 

In [14]:
len(data)

150600